In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

csv_train = pd.read_csv("../../data/train_data/Simon_train_3.csv")
df_train = pd.DataFrame(csv_train)

df_time_offset = df_train["Timestamp"][0]
df_train["Timestamp"] = df_train["Timestamp"] - df_time_offset
df_train

In [ ]:
POSE_MAP= {
    "rett": 0,
    "fram": 1,
    "fram-hoyre": 2,
    "hoyre": 3,
    "bak-hoyre": 4,
    "bak": 5,
    "bak-venstre": 6,
    "venstre": 7,
    "fram-venstre": 8
}

knn_train = struct_data(csv_f_name, annot_f_name, POSE_MAP)

In [ ]:
def get_timestamp_and_pose(filename):
    rows = []
    with open(filename, "r") as f:
        lines = f.readlines()
        offset = float(lines[1].strip().split(";")[1]) # This offset is the duration spent in the video before recording of data begun.
        for l in lines[1:]:
            sep_row = l.strip().split(";")
            finished_row = [round(float(x)-offset,2) for x in sep_row[1:3]]
            finished_row.append(POSE_MAP[sep_row[3].lower()])
            rows.append(finished_row)
    return rows

In [ ]:
f_name = "../../data/annotation/simon_train_3.txt"
stamped_poses = get_timestamp_and_pose(f_name)
print(stamped_poses)

df_stamped_poses = []
pose_index = 0
row_index = 0
for stamp in df_train["Timestamp"]:  
    pose_id = -1
    if stamp <= stamped_poses[-1][1]: 
        if stamp > stamped_poses[pose_index][1]:
            pose_index += 1
        pose_id = stamped_poses[pose_index][2]
        # If timestamp is in new index, but haven't reached the starting time of this interval yet, set pose_id to default value 5.
        if stamp >= stamped_poses[pose_index][0]:
            df_stamped_poses.append(pose_id)
        else:
            df_train = df_train.drop(row_index)
    row_index += 1

    # If timestamp of the data point exceeds last timestamp recorded by the annotation, append default -1 value
    #df_stamped_poses.append(pose_id)

#print(df_stamped_poses)

df_train["Pose"] = df_stamped_poses
#print(df[2400:2500][:])
print(len(df_train.index))
print(len(df_stamped_poses))
y_train = df_train["Pose"]
#df['Pose'].hist(bins=4)
sns.lineplot(data=df_train, x="Timestamp", y='Pose')

In [ ]:
x_train = df_train.drop(['Timestamp'],axis=1)
x_train = x_train.drop(['Pose'],axis=1)
x_train

In [ ]:
print(y_train)
print(len(y_train))

In [ ]:
csv_test = pd.read_csv("../../data/test_data/Simon_test_1.csv")
df_test = pd.DataFrame(csv_test)
df_time_offset = df_test["Timestamp"][0]
df_test["Timestamp"] = df_test["Timestamp"] - df_time_offset
df_test

In [ ]:
test_name = "../../data/annotation/Simon_test_1.txt"
stamped_poses = get_timestamp_and_pose(test_name)
print(stamped_poses)

df_stamped_poses = []
pose_index = 0
row_index = 0
drops = 0
for stamp in df_test["Timestamp"]:  
    pose_id = -1
    if stamp <= stamped_poses[-1][1] and stamp >= stamped_poses[0][0]:
        if stamp > stamped_poses[pose_index][1]:
            pose_index += 1
        pose_id = stamped_poses[pose_index][2]
        # If timestamp is in new index, but haven't reached the starting time of this interval yet, set pose_id to default value 5.
        if stamp >= stamped_poses[pose_index][0]:
            df_stamped_poses.append(pose_id)
        else:
            df_test = df_test.drop(row_index)
    else:
        df_test = df_test.drop(row_index)
    row_index += 1

    # If timestamp of the data point exceeds last timestamp recorded by the annotation, append default -1 value
    #df_stamped_poses.append(pose_id)

#print(df_stamped_poses)
print(len(df_test.index))
print("drops: {}".format(drops))
print(len(df_stamped_poses))
df_test["Pose"] = df_stamped_poses
y_test = df_stamped_poses
#print(df[2400:2500][:])

In [ ]:
x_test = df_test.drop(['Timestamp'],axis=1)
x_test = x_test.drop(['Pose'],axis=1)
x_test

In [ ]:
param_grid = [
    {'n_neighbours':[1,2,3,4,5,6], 'algorithm':['ball_tree', 'auto']}
]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knc = KNeighborsClassifier()
neigh = GridSearchCV(knc, param_grid)
neigh.fit(x_train,y_train)
predictions = neigh.predict(x_test)


In [ ]:
neigh.get_params()
accuracy_score(y_test, predictions)

In [ ]:
df_predict = pd.DataFrame({'x':df_test['Timestamp'],'y':predictions})
sns.lineplot(data=df_predict,x='x',y='y')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=True))

In [ ]:
sns.heatmap(confusion_matrix(y_test, predictions), cmap="YlGnBu", annot=True, fmt="d")

In [ ]:
number_of_corrects = 0
for num in range(len(predictions)):
    if(predictions[num] == y_test[num]):
        number_of_corrects += 1

print("number of correct: {}".format(number_of_corrects))
print("number of guesses: {}".format(len(predictions)))
print("% correct: {}".format(number_of_corrects/len(predictions)*100))